In [77]:
"""
4개의 보드를 부모로 사용 후 적합도를 검사해 2개의 보드를 부모삼아 새로운 보드를 생성, 총 4개의 자식을 생성
적합도는 뭉쳐서 그려진 캔디 그룹의 수가 낮을수록 (움직임이 유효한 캔디가 많을수록) 높음.
"""

import pygame
from pygame.locals import *
import random

# 게임 창 세팅
width = 400
height = 400
scoreboard_height = 25
window_size = (width, height + scoreboard_height)

# 7종류의 캔디
candy_colors = ['blue', 'green', 'orange', 'pink', 'purple', 'red', 'teal', 'yellow']

# 그려질 캔디의 크기
candy_width = 40
candy_height = 40
candy_size = (candy_width, candy_height)

#실제 게임에서 쓰일 보드
board = []

class Candy:

    def __init__(self, row_num, col_num, color):

        # 캔디 위치 저장
        self.row_num = row_num
        self.col_num = col_num

        self.color = candy_colors[color] #보드에 적혀있는대로 캔디에 색깔 할당
        image_name = f'swirl_{self.color}.png'
        self.image = pygame.image.load(image_name)
        self.image = pygame.transform.smoothscale(self.image, candy_size)
        self.rect = self.image.get_rect()
        self.rect.left = col_num * candy_width
        self.rect.top = row_num * candy_height

# 3개 이상의 캔디로 뭉친 그룹 찾기
def find_matches(board, candy, matches, direction):

    matches.add(candy)

    # 위 체크
    if direction == "up" and candy.row_num > 0:
        neighbor = board[candy.row_num - 1][candy.col_num]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 아래 체크
    if direction == "down" and candy.row_num < height / candy_height - 1:
        neighbor = board[candy.row_num + 1][candy.col_num]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 왼쪽 체크
    if direction == "left"  and candy.col_num > 0:
        neighbor = board[candy.row_num][candy.col_num - 1]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 오른쪽 체크
    if direction == "right" and candy.col_num < width / candy_width - 1:
        neighbor = board[candy.row_num][candy.col_num + 1]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    return matches

# 캔디 그룹이 3개이상이면 그 그룹을, 아니면 빈 세트를 반환
def match_three(board, candy, direction):

    matches = find_matches(board, candy, set(), direction)
    if len(matches) >= 3:
        return matches
    else:
        return set()


def MakeBoard():
  boards = []
  for i in range(4):
      boards.append([])
      for row_num in range(height // candy_height):
          boards[i].append([])
          for col_num in range(width // candy_width):
              candy = Candy(row_num, col_num, random.randint(0, 7))
              boards[i][row_num].append(candy)
  return boards

#변이
def Mutation(board):
    for row in board:
        for candy in row:
            if(random.randint(1, 100) == 1):
                candy.color = candy_colors[random.randint(0, 7)]
                image_name = f'swirl_{candy.color}.png'
                candy.image = pygame.image.load(image_name)
                candy.image = pygame.transform.smoothscale(candy.image, candy_size)
    return board

#교차
def crossover(parent):
    row = random.randint(0, 10)
    col = random.randint(0, 10)
    child = [[] for i in range(10)]
    for i in range(row):
        for j in range(col):
            child[i].append(parent[0][i][j])
        for j in range(10 - col):
            child[i].append(parent[1][i][j + col])

    for i in range(10 - row):
        for j in range(col):
            child[i + row].append(parent[1][i + row][j])
        for j in range(10 - col):
            child[i + row].append(parent[0][i + row][j + col])

    return Mutation(child)

#룰렛 휠
def wheel(boards, fitness):
    sum = 0
    parent = []
    for i in fitness:
        sum += i
    for i in range(2):
        rand = random.randint(1, sum)
        for j in range(4):
            if(rand <= fitness[j]):
                parent.append(boards[j])
                break
            else:
                rand -= fitness[j]
    return crossover(parent)


#보드의 적합도 검사
def selection(board):
    unfitness = 0
    for row in board:
        for candy in row:
            tmp = match_three(board, candy, "up")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "down")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "left")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "right")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
    return unfitness

#유전자 알고리즘으로 보드 생성
gene = 0
noUse = 0
sum = 0
boards = MakeBoard()
selected_board = 4
test = 100
for k in range(test):
  while True:
    fit = []
    for i in range(4):
      fit.append(100 - selection(boards[i]))

    for i in range(4):
        if(fit[i] == 100):
          selected_board = i

    if(selected_board < 4):
        board = boards[selected_board]
        print(gene)
        sum += gene
        if(gene == 0):
          noUse += 1
        gene = 0
        boards = MakeBoard()
        board = []
        selected_board = 4
        break

    for board in boards:
      board = wheel(boards, fit)

    gene += 1

print("NnoUse: ", noUse/test * 100, "%")
print("sum: ", sum/test)

5
1
41
0
50
0
7
0
9
49
60
0
0
30
84
30
35
84
64
0
17
0
26
1
9
84
55
13
0
0
23
0
3
21
2
31
0
45
39
95
2
0
17
53
0
0
46
177
0
16
0
0
0
23
8
0
13
25
37
20
0
43
67
1
0
72
48
76
0
0
0
133
0
31
0
109
0
0
61
14
0
0
70
141
37
150
0
5
0
0
1
15
262
48
13
4
74
9
0
2
NnoUse:  34.0 %
sum:  29.36
